In [29]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.stats import poisson, lognorm, weibull_min
import torch
import igraph as ig
from pathlib import Path
import pickle
from src.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR
from src.models.dynamicHotNet import HoT_GNN

In [30]:
# 加载数据
with open(PROCESSED_DATA_DIR / "temporal_hypergraph.pkl", "rb") as f:
    data = pickle.load(f)
temporal_hyperedges = data['temporal_hyperedges']
dynamic_features = data['dynamic_features']
disaster_exposure = data['disaster_exposure']
X_n = data['X_n']
X_e = data['X_e']
A_tilde = data['A_tilde']
B1_tilde = data['B1_tilde']
L1_tilde = data['L1_tilde']

In [31]:
n_nodes, n_edges, n_timesteps = 9168, 11667, 600
F_nodes, F_edges = X_n.shape[1], X_e.shape[1]

In [32]:
# 检查维度
assert dynamic_features.shape == (n_nodes, n_timesteps, F_nodes), f"Expected dynamic_features shape {(n_nodes, n_timesteps, F_nodes)}, got {dynamic_features.shape}"

In [33]:
# 1. 生成灾害事件
def generate_disaster_events(n_timesteps, seed=42):
    np.random.seed(seed)
    events = []
    earthquake_rate = 0.01 * n_timesteps / (50 * 12)
    n_earthquakes = poisson.rvs(earthquake_rate * n_timesteps)
    earthquake_times = np.random.uniform(0, n_timesteps, n_earthquakes)
    earthquake_magnitudes = lognorm.rvs(s=0.5, scale=np.exp(6), size=n_earthquakes)
    flood_rate = 0.05 * n_timesteps / (50 * 12)
    n_floods = poisson.rvs(flood_rate * n_timesteps)
    flood_times = np.random.uniform(0, n_timesteps, n_floods)
    flood_depths = lognorm.rvs(s=0.5, scale=np.exp(0), size=n_floods)
    hurricane_rate = 0.02 * n_timesteps / (50 * 12)
    n_hurricanes = poisson.rvs(hurricane_rate * n_timesteps)
    hurricane_times = np.random.uniform(0, n_timesteps, n_hurricanes)
    hurricane_speeds = weibull_min.rvs(c=2, scale=20, size=n_hurricanes)
    for t, mag in zip(earthquake_times, earthquake_magnitudes):
        events.append(('earthquake', t, mag))
    for t, depth in zip(flood_times, flood_depths):
        events.append(('flood', t, depth))
    for t, speed in zip(hurricane_times, hurricane_speeds):
        events.append(('hurricane', t, speed))
    events.sort(key=lambda x: x[1])
    return events

disaster_events = generate_disaster_events(n_timesteps)

In [34]:
print(len(disaster_events))
disaster_events

44


[('earthquake', np.float64(12.350696577481468), np.float64(254.1586252942617)),
 ('hurricane', np.float64(15.251476046457114), np.float64(11.698552113608882)),
 ('hurricane', np.float64(18.85751141204055), np.float64(32.588035141018395)),
 ('flood', np.float64(20.633112669131037), np.float64(0.8144912006756959)),
 ('flood', np.float64(27.870247631998634), np.float64(0.5100028073451038)),
 ('flood', np.float64(39.03095579116771), np.float64(1.069524803889581)),
 ('flood', np.float64(58.60326840383032), np.float64(0.9016012362786295)),
 ('hurricane', np.float64(64.73485619598267), np.float64(8.385901466152887)),
 ('flood', np.float64(73.2229409068673), np.float64(1.7329471047237062)),
 ('flood', np.float64(83.6963163912251), np.float64(4.011465038183434)),
 ('flood', np.float64(102.31447421237492), np.float64(0.5682032144422158)),
 ('flood', np.float64(119.80426929501584), np.float64(0.4530978797419071)),
 ('flood', np.float64(155.26798896001014), np.float64(0.6381997417908131)),
 ('floo

In [35]:
# 2. 生成交互标签
def generate_interaction_labels(events, disaster_exposure, seed=42):
    np.random.seed(seed)
    interactions = []
    for event in events:
        disaster_type, t, intensity = event
        if disaster_type == 'earthquake' and intensity > 6:
            prob = 0.8 if intensity > 7 else 0.6
            affected_nodes = np.where(disaster_exposure[:, 0] > np.percentile(disaster_exposure[:, 0], 90))[0]
            interactions.append(('earthquake-liquefaction', t, prob, affected_nodes))
        elif disaster_type == 'flood':
            hurricane_events = [e for e in events if e[0] == 'hurricane' and abs(e[1] - t) < 1]
            if hurricane_events:
                prob = 0.9
                affected_nodes = np.where(disaster_exposure[:, 1] > np.percentile(disaster_exposure[:, 1], 90))[0]
                interactions.append(('flood-hurricane', t, prob, affected_nodes))
    return interactions

interaction_labels = generate_interaction_labels(disaster_events, disaster_exposure)

In [36]:
# 3. 训练HoT_GNN
model = HoT_GNN(n_nodes=9168, node_features=F_nodes, edge_features=F_edges, node_hidden=[32, 16, 8], edge_hidden=[16, 8, 1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

# GPU支持（可选）
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)
X_e = X_e.to(device)
A_tilde = A_tilde.to(device)
L1_tilde = L1_tilde.to(device)
B1_tilde = B1_tilde.to(device)

In [37]:
hyperedge_labels = [(t, nodes, prob) for _, t, prob, nodes in interaction_labels]
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    total_loss = 0
    for t in range(n_timesteps):
        X_n_t = torch.tensor(dynamic_features[:, t, :], dtype=torch.float32).to(device)
        outputs = model(X_n_t, X_e, A_tilde, L1_tilde, B1_tilde, temporal_hyperedges[t])
        if outputs['hyperedge_prob'] is not None:
            target = [prob for t_label, _, prob in hyperedge_labels if abs(t - t_label) < 0.5]
            if target and len(target) == len(outputs['hyperedge_prob']):
                target = torch.tensor(target, dtype=torch.float32).to(device)
                loss = criterion(outputs['hyperedge_prob'], target)
                total_loss += loss
    if total_loss > 0:
        total_loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item() if isinstance(total_loss, torch.Tensor) else 0}")

Epoch 0, Loss: 0


KeyboardInterrupt: 

In [ ]:
# 4. 预测初始失效
def predict_initial_failure(events, disaster_exposure, model, dynamic_features, X_e, A_tilde, B1_tilde, L1_tilde, device):
    failure_probs = np.zeros((n_nodes, n_timesteps))
    model.eval()
    with torch.no_grad():
        for t in range(n_timesteps):
            X_n_t = torch.tensor(dynamic_features[:, t, :], dtype=torch.float32).to(device)
            outputs = model(X_n_t, X_e, A_tilde, L1_tilde, B1_tilde)
            failure_probs[:, t] = outputs['node_prob'].cpu().numpy()
    return failure_probs

initial_failure_probs = predict_initial_failure(disaster_events, disaster_exposure, model, dynamic_features, X_e, A_tilde, B1_tilde, L1_tilde, device)

In [ ]:
# 5. 更新时间超图
def update_temporal_hypergraph(temporal_hyperedges, initial_failure_probs, interaction_labels):
    updated_hyperedges = temporal_hyperedges.copy()
    for t in range(n_timesteps):
        failed_nodes = np.where(initial_failure_probs[:, t] > 0.5)[0]
        if len(failed_nodes) > 1:
            updated_hyperedges[t].append(failed_nodes.tolist())
        for inter_type, inter_t, prob, nodes in interaction_labels:
            if abs(inter_t - t) < 0.5 and prob > 0.7:
                updated_hyperedges[t].append(nodes.tolist())
    return updated_hyperedges

updated_hyperedges = update_temporal_hypergraph(temporal_hyperedges, initial_failure_probs, interaction_labels)


In [ ]:
# 保存结果
with open(PROCESSED_DATA_DIR / "disaster_simulation.pkl", "wb") as f:
    pickle.dump({
        'disaster_events': disaster_events,
        'interaction_labels': interaction_labels,
        'initial_failure_probs': initial_failure_probs,
        'updated_hyperedges': updated_hyperedges
    }, f)

print("灾害模拟完成，保存至 disaster_simulation.pkl")